In [1]:
%load_ext autoreload
%autoreload

import dut_func as dut
import serial
import time
import struct
import numpy as np
# from bitarray import bitarray
from bitstring import BitArray

from misc import *

# dut.drv.serUSB = 'COM19'
drv = dut.drv

ser = serial.Serial('COM6', 9600, timeout=1)
dut.connect(ser)



In [213]:
dut.disconnect()

Serial port disconnected.


In [2]:
dut.pads_defaults()

In [79]:
dut.dac_set('PLANE_VPP', 0.22)

DAC initialized to a span from -5 V to 5 V


In [448]:
voltages = drv.pic_adc_read() *2
voltages

array([0.        , 0.49802198, 0.07413919, 0.48190476, 5.10271062])

Set up various DAC channels

In [437]:
dut.dac_set('DAC_VREF_HI_CMP', 3.92)

In [438]:
dut.dac_set('P_VREF_SH', 2.42)

In [439]:
dut.dac_set('P_VAGC_1', 3.9)

In [440]:
dut.dac_set('P_VAGC_0', 1)

In [441]:
dut.dac_set('P_TVDD', 4.92)

In [442]:
dut.dac_set('P_TVDD', 1.)

In [443]:
dut.dac_set('P_VREF_TIA', 0.42)

In [444]:
dut.dac_set('DAC_VREF_ARRAY', 0.42)

In [445]:
dut.dac_set('DAC_VP_PAD', 2.5)

In [446]:
dut.dac_set('P_AMP_VREF', 2.5)

In [447]:
dut.dac_set('P_AMP_INPUT', 2.5)

________________

Assert Power GOOD

In [449]:
drv.gpio_pin_set(*PIC_PINS['PWR_GOOD'])

Pulse RESET chip with a rising edge

In [450]:
portName, pinPos = PIC_PINS['NRESET_FULL_CHIP']
drv.gpio_pin_reset(portName, pinPos)
time.sleep(0.1) # want to delay 1us
drv.gpio_pin_set(portName, pinPos)

Send a scan chain data

In [451]:
data = bytes([0x10, 0x20, 0x0c, 0x10, 0x20, 0x01, 0x02])
drv.spi_serial_write(2, data)

In [18]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(1, data)

b'\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04'


In [173]:
for _ in range(20):
    data = bytes([0x10, 0x20, 0x0c, 0x00, 0x20, 0x01, 0x02] * 20)
    drv.spi_serial_write(1, data)

In [89]:
data

b'\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02\x10 \x0c\x00 \x01\x02'

In [20]:
dut.disconnect()

Serial port disconnected.


In [36]:
# NOTE: first power on requires physical switch actions on DUT board as well as PIC commands

# FIRST: SWITCH microntroller VDD power ON (i.e. power up microcontroller)
time.sleep(1) # want to delay 1us
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
#dut.dac_init()

In [19]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b000_10000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b00000001, 0b00000010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [40]:
drv.clk_start('CK_ARRAY')
drv.clk_start('ADC_CK')

In [24]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
time.sleep(1e-6) # want to delay 1us
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [86]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])

In [87]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

In [86]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [80]:
drv.gpio_pin_set(*PIC_PINS['PWR_GOOD'])

In [85]:
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE2'])

In [25]:
drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])

In [26]:
drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_SH'])

In [27]:
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
# dac_set('P_ADC_EXT_TEST_IN', 1.5)       # voltage = 1.5 as an example, I wonder whether we should put it as a parameter of the function
time.sleep(1e-6)        # delay(t_sel_ext or t_ext_inp), min = 2CK
drv.gpio_pin_set(*PIC_PINS['ADC_SEL_EXTERNAL'])
#time.sleep(1e-6)       # delay(t_en_overide_sh), min = 0CK
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])
time.sleep(1e-6)        # delay(t_fire_sh), min = 3CK
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])
time.sleep(1e-6)        # delay(3 P_ADC_CK periods)
# drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_SH'])
while True:
    if drv.gpio_pin_is_high(*PIC_PINS['ADC_DONE']):
        break
#download_fifo()
dut.pads_defaults()
#reset_dpe()

KeyboardInterrupt: 

In [41]:
dut.pads_defaults()

In [29]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])

In [46]:
drv.gpio_pin_set(*PIC_PINS['NRESET_FULL_CHIP'])

In [31]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])

WEDNESDAY ADC DONE TEST AND DOCUMENTATION

EXECUTE IN ORDER

In [434]:
dut.dac_init()

DAC initialized to a span from -5 V to 5 V


In [435]:
dut.vrefs_off()

In [432]:
drv.gpio_pin_reset(*PIC_PINS['PWR_GOOD'])

In [47]:
drv.gpio_pin_set(*PIC_PINS['PWR_GOOD'])

In [48]:
dut.dac_set('PLANE_VPP', 0.22)

In [49]:
dut.dac_set('DAC_VREF_HI_CMP', 3.92)

In [50]:
dut.dac_set('DAC_VREF_ARRAY', 0.42)

In [51]:
dut.dac_set('P_VREF_SH', 2.42)

In [52]:
dut.dac_set('DAC_VP_PAD', 2.42)

In [53]:
dut.dac_set('P_VAGC_1', 3.9)

In [54]:
dut.dac_set('P_VAGC_0', 1)

In [55]:
dut.dac_set('P_TVDD', 4.92)

In [56]:
dut.dac_set('P_VREF_TIA', 0.42)

In [137]:
dut.dac_set('P_ADC_EXT_TEST_IN', 1.5)

*******************************************************

FOR ADC OPERATION, PROCEED FROM HERE (07/24/2019) 16:40pm - assumes PWR good asserted

In [5]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()

In [431]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])


In [7]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b000_10000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b00000001, 0b00000010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [430]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])

In [62]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(1, data)

b'\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04'
b'\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04\xc10L\x13\x04'


In [75]:
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])

In [76]:
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

In [65]:
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<2>'])

In [141]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [142]:
drv.gpio_pin_set(*PIC_PINS['ADC_SEL_EXTERNAL'])

In [143]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])

In [144]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

In [145]:
drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_SH'])

In [39]:
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE2'])

In [42]:
dut.pads_defaults()

In [43]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])

BEGIN FIFO TESTING

In [166]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

In [189]:
addr=np.binary_repr(3)
print(addr)

drv.gpio_adc_fifo_en_write(addr) # not working yet
drv.gpio_pin_is_high(*PIC_PINS['ADC_FIFO_EN<0>'])
#drv.gpio_pin_is_high(*PIC_PINS['ADC_FIFO_EN<1>'])
#drv.gpio_pin_is_high(*PIC_PINS['ADC_FIFO_EN<2>'])
#drv.gpio_pin_is_high(*PIC_PINS['ADC_FIFO_EN<3>'])

11


True

In [190]:
voltages = np.arange(0.5, 4.5, 0.5)

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

for V_adc in voltages:
    
    dut.dac_set('P_ADC_EXT_TEST_IN', V_adc)    
    
    time.sleep(1e-6)
    drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_SH'])
    drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
    drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])
    drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

    StrobeNum = 15
    for strobe in range(StrobeNum):
        drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
        drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
        #print(strobe)

    print(V_adc)
    data = drv.gpio_adc_read()
    print(f'{data:013b}')
        

0.5
0100000000000
1.0
0100010001000
1.5
0100100100000
2.0
0100110110110
2.5
0101001001111
3.0
0101011011111
3.5
0101101111111
4.0
0101111111111


0101111111111
101111111111


In [13]:
def adc2volt(data):
    V_HI = 4.0
    V_LO = 0.5
    value = data & 0x3ff
    voltage = (V_HI-V_LO) * value / 1023 + V_LO
    return voltage

****************************************



Testing TIA configuration mapping 07/25/19

In [25]:
# Testing TIA config column mapping
#11000_00001
# test config: 0b1100_0000, 0b0111_0000, 0b0010_1100, 0b0001_0011, 0b0000_1000
# data = bytes([0b1100_0000, 0b1011_0000, 0b0100_1100, 0b0010_0011, 0b0001_0000]*24)


drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])

# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0100, 0b0011_0000, 0b0001_1100, 0b0000_0111, 0b0000_0001]*24)
print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0000, 0b0111_0000, 0b0010_1100, 0b0001_0011, 0b0000_1000]*24)
print(data)
drv.spi_serial_write(1, data)

time.sleep(1e-6)
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])



b'\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01'
b'\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01\xc40\x1c\x07\x01'


In [26]:
values = drv.spi_serial_write_and_read(1, data)
for v in values:
    print(f'{v:08b}')

11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
00110000
00011100
00000111
00000001
11000100
0

In [27]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])


In [28]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])


data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )


0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V
0000000000000	0.500 V


LOAD VECTORS

In [151]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])

In [158]:
drv.gpio_pin_set(*PIC_PINS['NRESET_FULL_CHIP'])

In [159]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b000_10000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b00000001, 0b00000010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [178]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()

In [161]:
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [395]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0100, 0b0011_0000, 0b1000_1100, 0b0001_0011, 0b0000_0010]*24)
data = BitArray('0b1100000010'*96).bytes

print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(1, data)

time.sleep(1e-6)
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])


b'\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02'
b'\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02\xc0\xb0,\x0b\x02'


In [63]:
values = drv.spi_serial_write_and_read(1, data)
for v in values:
    print(f'{v:08b}')
time.sleep(1e-6)
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
00110000
10001100
00010011
00000010
11000100
0

In [177]:
#drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
#dut.pads_defaults()
#drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [167]:
roc = [0 ,0, 0]
# data= 0b1111111111111111
data= 0x0000
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [168]:
roc = [1 ,1, 1]
# data= 0b1111111111111111
data= 0x0000
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

READ TESTING


In [66]:
dut.pads_defaults()

In [67]:
#drv.gpio_pin_set(*PIC_PINS['DPE_INTERNAL_EN'])

# EXETERNALLY TRIGGERED READ
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN']) 

In [68]:
drv.gpio_pin_set(*PIC_PINS['READ_DPE'])

In [69]:
drv.gpio_pin_set(*PIC_PINS['READ_BIT'])

In [ ]:
#drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<0>'])
#drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<1>'])
#drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<2>'])

In [70]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [71]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_TIA'])

In [72]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_COLUMN_T'])

In [73]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_PULSE'])

In [74]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

In [150]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000000	0.500 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V
0111000000000	2.252 V


**********************************************************************

MOVING ON FOR NOW - TIA BITS STILLL NOT READ OUT BUT CLEAR CORRELATION BETWEEN LOADED GAIN SETTINGS AND ADC OUTPUT READ

TIA CALIBRATION TIME!

In [926]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])

In [927]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])

In [928]:
drv.gpio_pin_set(*PIC_PINS['NRESET_FULL_CHIP'])

In [929]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b000_10000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b00000001, 0b00000010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [930]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [931]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION
#data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)

data = BitArray('0b1100010000'*96).bytes
print(data)
drv.spi_serial_write(0, data)
print(data)
drv.spi_serial_write(1, data)

time.sleep(1e-5)
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

b'\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10'
b'\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10'


In [932]:
# LOAD ONE COLUMN ON ONE ARRAY (NOTE: THIS ASSUMES YOU"VE PREVIOUSLY LOADED ALL ZEROS)

drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<2>'])

drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])

drv.gpio_row_col_bank_write(0b0001)
drv.gpio_row_col_data_write(0b1000_0000_0000_0000)
time.sleep(1e-5)
drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
time.sleep(1e-5)
drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

In [965]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [966]:
drv.gpio_pin_set(*PIC_PINS['WRITE_SEL_EXT'])
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])                 
time.sleep(5e-6)   
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE2'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE0'])
time.sleep(5e-6)        # delay(t_cal_start), min = 2TCK
drv.gpio_pin_set(*PIC_PINS['COL_WRITE_CONNECT'])
time.sleep(5e-6)        # delay(t_opamp), min = 500ns
drv.gpio_pin_set(*PIC_PINS['CONNECT_TIA'])
#drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

In [967]:
# Before setting this, we're using an external current source to apply to the board!
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])
# Then see that ADC_DONE goes high and strobe out the FIFO

In [968]:
dut.pads_defaults()

In [969]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()


0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000100010	0.616 V
0000000000010	0.507 V
0000000000010	0.507 V
0000110010010	1.875 V
0001101111111	3.562 V
0000000000010	0.507 V
0000011000010	1.164 V
0000000000010	0.507 V
0000000010010	0.562 V
0001011001111	2.960 V
0000000000010	0.507 V
0000000000010	0.507 V

0000000000010	0.507 V
0000110010010	1.875 V
0000000000010	0.507 V
0000000000010	0.507 V
0001111111111	4.000 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0001110111111	3.781 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V

0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0001101111111	3.562 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
00010110

In [692]:
#turn on all columns
roc = [1 ,1, 1]
# data= 0b1111111111111111
data= 0x1111
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [131]:
dut.dac_set('DAC_SCHOTTKY', -0.2)

In [204]:
dut.dac_set('P_VREF_SH', 2.42)

In [306]:
dut.dac_set('P_TVDD', -0.08)

In [393]:
from bitstring import BitArray
conf_bits = BitArray('0b1011000001'*96).bytes
conf_bits

b'\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1\xb0l\x1b\x06\xc1'

In [352]:
conf_bytes = []
for bit in conf_bits.bytes:
    print(f'{bit:08b}', end='_')

10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_01101100_00011011_00000110_11000001_10110000_0

INTERNAL READ TIME!

In [453]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b0001_0000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b0000_0001, 0b0000_0010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [610]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [614]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
#1kohm	5kohm	30kohm	200kohm	1Mohm	cm	C<1>	C<0>	TIA en	S/H en
#data = BitArray('0b1100000001'*96).bytes
data = BitArray('0b11000000011100000010110000000111000001001100000010110000100011000000011100010000'*12).bytes

print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(1, data)

time.sleep(1e-6)
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

b'\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10'
b'\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10'


In [615]:
roc = [0 ,0, 0]
# data= 0b1111111111111111
data= 0x1111
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [616]:
roc = [1 ,1, 1]
# data= 0b1111111111111111
data= 0x0000
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [617]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [618]:
drv.gpio_pin_set(*PIC_PINS['DPE_INTERNAL_EN'])
drv.gpio_pin_set(*PIC_PINS['AGC_INTERNAL_EN'])

In [619]:
drv.gpio_pin_set(*PIC_PINS['READ_BIT'])

In [620]:
drv.gpio_pin_set(*PIC_PINS['READ_DPE'])

In [621]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [622]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_TIA'])

In [623]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_COLUMN_T'])

In [624]:
#drv.gpio_pin_set(*PIC_PINS['DPE_PULSE'])
drv.gpio_pin_set(*PIC_PINS['AGC_PULSE'])

In [625]:
dut.pads_defaults()

In [626]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

0110000000010	0.507 V
0110011000010	1.164 V
0110100100010	1.492 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001000010	0.726 V
0110000000010	0.507 V
0110000000010	0.507 V
0110100110010	1.547 V
0110000000010	0.507 V
0110000100010	0.616 V
0110000000010	0.507 V
0110000100010	0.616 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000100010	0.616 V
0110000000010	0.507 V
0110011100010	1.273 V
0110100010010	1.437 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001100010	0.835 V
0110100000010	1.383 V

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110010000010	0.945 V
0110000000010	0.507 V
0110001100010	0.835 V
0110000000010	0.507 V
0110000000010	0.507 V
01100010

In [551]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110011101010	1.301 V
0110100001010	1.410 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001000010	0.726 V
0110000000010	0.507 V
0110000000010	0.507 V
0110100001010	1.410 V
0110000000010	0.507 V
0110000100010	0.616 V
0110000000010	0.507 V
0110000100010	0.616 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V


In [552]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000100010	0.616 V
0110000000010	0.507 V
0110100100010	1.492 V
0110001100010	0.835 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001000010	0.726 V
0110011100110	1.287 V


In [553]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001000010	0.726 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110010000010	0.945 V
0110000000010	0.507 V
0110001000010	0.726 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001000010	0.726 V
0110000000010	0.507 V
0110000000010	0.507 V


In [554]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110001000010	0.726 V
0110000000010	0.507 V
0110000000010	0.507 V
0110010000010	0.945 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001000010	0.726 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V


In [555]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110100000010	1.383 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110010000010	0.945 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001100010	0.835 V
0110010000010	0.945 V
0110000100010	0.616 V
0110010000010	0.945 V
0110001100010	0.835 V
0110001000010	0.726 V


In [556]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110001000010	0.726 V
0110000000010	0.507 V
0110000000010	0.507 V
0110010000010	0.945 V
0110000000010	0.507 V
0110000100010	0.616 V
0110001100110	0.849 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001100010	0.835 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V


In [557]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000100010	0.616 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001100010	0.835 V
0110000000010	0.507 V
0110000000010	0.507 V
0110100000010	1.383 V
0110000000010	0.507 V
0110010100010	1.054 V
0110010100010	1.054 V


In [558]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110101010010	1.656 V
0110001000010	0.726 V
0110001000010	0.726 V
0110011100010	1.273 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V


In [559]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110010000010	0.945 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110001000010	0.726 V
0110010000010	0.945 V
0110101000010	1.602 V


In [560]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110001100010	0.835 V
0110000000010	0.507 V
0110000000010	0.507 V
0110010100010	1.054 V
0110001100010	0.835 V
0110000000010	0.507 V
0110010100010	1.054 V
0110000000010	0.507 V
0110001000010	0.726 V
0110001100010	0.835 V
0110000000010	0.507 V
0110100000010	1.383 V
0110001000010	0.726 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V


In [561]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110010000010	0.945 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110011100010	1.273 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0111000011010	2.341 V
0110011100010	1.273 V
0110000000010	0.507 V
0110000100010	0.616 V
0110010000010	0.945 V
0110000000010	0.507 V


In [562]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )

0110000000010	0.507 V
0110011100010	1.273 V
0110011000010	1.164 V
0110000000010	0.507 V
0110010100010	1.054 V
0110011100010	1.273 V
0110000000010	0.507 V
0110000000010	0.507 V
0110011101010	1.301 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V


In [616]:
roc = [1 ,1, 1]
# data= 0b1111111111111111
data= 0x0000
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])